## Importando Bibliotecas

In [36]:
import tensorflow as tf 
from tensorflow import keras
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Input, Dense
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from keras.optimizers import Adam
from keras.models import Model
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

## Pre-processamento dos dados
**Carregando os dados de treino e teste:**

In [2]:
train_images = pd.read_csv('../fashion-mnist_train.csv')
test_images = pd.read_csv('../fashion-mnist_test.csv')

**Separando as labels do conjuntos:**

In [3]:
train_labels=train_images.loc[:, ['label']]
train_images=train_images.drop(['label'], axis=1)
test_labels=test_images.loc[:, ['label']]
test_images=test_images.drop(['label'], axis=1)

**Normalizando os data-sets:**

In [4]:
train_images = train_images / 255.0
test_images = test_images / 255.0


----------------------
## Construção do modelo base
**Esta rede será utilizada posteriomente para comparações entre os modelos de dimensionalidade reduzida utilizando PCA e autoencoder.**

**Definição da estrutura do modelo por meio do Keras Sequential:**

In [5]:
model = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(784)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 59,850
Trainable params: 59,850
Non-trainable params: 0
_________________________________________________________________


**Compilação do modelo:**

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**Treino do modelo:**

In [7]:
model.fit(train_images, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 6s 135us/sample - loss: 0.5268 - accuracy: 0.8111 - val_loss: 0.4501 - val_accuracy: 0.8418
Epoch 2/5
48000/48000 [==============================] - 4s 91us/sample - loss: 0.3909 - accuracy: 0.8587 - val_loss: 0.3902 - val_accuracy: 0.8593
Epoch 3/5
48000/48000 [==============================] - 4s 89us/sample - loss: 0.3502 - accuracy: 0.8720 - val_loss: 0.3735 - val_accuracy: 0.8652
Epoch 4/5
48000/48000 [==============================] - 4s 93us/sample - loss: 0.3298 - accuracy: 0.8798 - val_loss: 0.3779 - val_accuracy: 0.8675
Epoch 5/5
48000/48000 [==============================] - 4s 91us/sample - loss: 0.3109 - accuracy: 0.8846 - val_loss: 0.3681 - val_accuracy: 0.8671


**Verificação dos resultados do modelo:**

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 42us/sample - loss: 0.3600 - accuracy: 0.8659
Model - 3 layers - test loss: 36.00049532532692
Model - 3 layers - test accuracy: 86.58999800682068


-----------------------------
## Redução de Dimensionalidade Usando Principal component analysis (PCA)

**Definição do número de componentes:**

In [9]:
pca = PCA(n_components=3)

**Construção dos novos datasets de treino e de teste com dimensões reduzidas usando o PCA:**

In [10]:
train_images_r = pca.fit(train_images).transform(train_images)
test_images_r = pca.fit(test_images).transform(test_images)

**Alteração do tipo dos dados de numpy array para pandas dataframe:**

In [11]:
train_images_r = pd.DataFrame(data=train_images_r)
test_images_r= pd.DataFrame(data=test_images_r)

**Definição da estrutura do modelo por meio do Keras Sequential que receberá os dados com dimensões reduzidas pelo PCA:**

In [12]:
model_r = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(3)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model_r.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                256       
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 9,866
Trainable params: 9,866
Non-trainable params: 0
_________________________________________________________________


**Compilação do modelo:**

In [13]:
model_r.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**Treino do modelo:**

In [14]:
model_r.fit(train_images_r, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 56us/sample - loss: 0.9873 - accuracy: 0.5931 - val_loss: 0.9187 - val_accuracy: 0.6105
Epoch 2/5
48000/48000 [==============================] - 3s 52us/sample - loss: 0.8963 - accuracy: 0.6257 - val_loss: 0.8929 - val_accuracy: 0.6267
Epoch 3/5
48000/48000 [==============================] - 3s 61us/sample - loss: 0.8765 - accuracy: 0.6345 - val_loss: 0.8772 - val_accuracy: 0.6302
Epoch 4/5
48000/48000 [==============================] - 3s 70us/sample - loss: 0.8671 - accuracy: 0.6386 - val_loss: 0.8839 - val_accuracy: 0.6298
Epoch 5/5
48000/48000 [==============================] - 4s 82us/sample - loss: 0.8613 - accuracy: 0.6412 - val_loss: 0.8564 - val_accuracy: 0.6487


**Verificação dos resultados do modelo:**

In [15]:
test_loss, test_acc = model_r.evaluate(test_images_r, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 43us/sample - loss: 0.8670 - accuracy: 0.6335
Model - 3 layers - test loss: 86.70160306930542
Model - 3 layers - test accuracy: 63.349997997283936


------------------------------------------------------

## Redução de Dimensionalidade Usando Autoencoders

In [16]:
# this is the size of our encoded representations
encoding_dim = 2
input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
encoder = Model(input_img, encoded)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input_img, decoded)
encoded_input = Input(shape=(encoding_dim,))
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [17]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [18]:
#REMOVER troque o 'adadelta' pelo 'adam'
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [19]:
autoencoder.fit(train_images, train_images,epochs=5,shuffle=True,validation_data=(test_images, test_images))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 135us/step - loss: 0.4458 - val_loss: 0.4148
Epoch 2/5
60000/60000 [==============================] - 8s 125us/step - loss: 0.4055 - val_loss: 0.3956
Epoch 3/5
60000/60000 [==============================] - 7s 116us/step - loss: 0.3903 - val_loss: 0.3842
Epoch 4/5
60000/60000 [==============================] - 7s 116us/step - loss: 0.3818 - val_loss: 0.3785
Epoch 5/5
60000/60000 [==============================] - 8s 135us/step - loss: 0.3779 - val_loss: 0.3763


In [20]:
encoded_imgs_train = encoder.predict(train_images)
encoded_imgs_test = encoder.predict(test_images)


In [21]:
train_images_a = pd.DataFrame(data=encoded_imgs_train)
test_images_a =pd.DataFrame(data=encoded_imgs_test)

In [22]:
model_r = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(2)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model_r.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                192       
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 9,802
Trainable params: 9,802
Non-trainable params: 0
_________________________________________________________________


In [23]:
model_r.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
model_r.fit(train_images_a, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 70us/sample - loss: 1.2556 - accuracy: 0.4880 - val_loss: 1.1490 - val_accuracy: 0.5382
Epoch 2/5
48000/48000 [==============================] - 3s 65us/sample - loss: 1.1479 - accuracy: 0.5324 - val_loss: 1.1537 - val_accuracy: 0.5361
Epoch 3/5
48000/48000 [==============================] - 3s 61us/sample - loss: 1.1335 - accuracy: 0.5403 - val_loss: 1.1280 - val_accuracy: 0.5502
Epoch 4/5
48000/48000 [==============================] - 3s 53us/sample - loss: 1.1272 - accuracy: 0.5427 - val_loss: 1.1257 - val_accuracy: 0.5458
Epoch 5/5
48000/48000 [==============================] - 3s 64us/sample - loss: 1.1236 - accuracy: 0.5449 - val_loss: 1.1177 - val_accuracy: 0.5472


In [25]:
test_loss, test_acc = model_r.evaluate(test_images_a, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 30us/sample - loss: 1.1248 - accuracy: 0.5441
Model - 3 layers - test loss: 112.4824024963379
Model - 3 layers - test accuracy: 54.409998655319214


------------------------------------------------------

## Clustering com K-means

Optamos pela utilização das imagens reduzidas por meio do PCA, já que obtiveram por meio deste modelo os melhores resultados de predição na rede neural.

In [26]:
train_images_k = train_images_r
test_images_k = test_images_r

In [27]:
kmean_model = KMeans(n_clusters=10, random_state=10)
y_km = kmean_model.fit(train_images_k)
labels = kmean_model.labels_

In [28]:
cluster_centroids = kmean_model.cluster_centers_

In [29]:
#silhouette_avg = silhouette_score(X = train_images_k, labels = train_labels.values.ravel(), random_state=10)

In [34]:
silhouette_avg = silhouette_score(X = train_images_k, labels = train_labels.values.ravel(), random_state=10)

In [39]:
metrics.homogeneity_score(train_labels.values.ravel(), labels)

0.4536478741464148

In [41]:
metrics.v_measure_score(train_labels.values.ravel(), labels, beta=1.0)

0.45694727123110823

---------------------------------
## Clustering com o DBScan
